# ZRP Zip Code Only
The purpose of this notebook is to illustrate how to use the `zrp` package to generate race/ethnicity proxies that are based on zip code only. 


In [1]:
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi=False

In [2]:
import pandas as pd
import sys
import os
import re
import warnings
from zrp.prepare.utils import load_file, load_json, make_directory
from os.path import join, expanduser, dirname

warnings.filterwarnings(action='ignore')
home = expanduser('~')

In [3]:
import pkg_resources
print("ZRP version:", pkg_resources.get_distribution('zrp').version)

ZRP version: 0.3.3


## Load sample data for prediction
Load processed list of New Jersey Mayors downloaded from https://www.nj.gov/dca/home/2022mayors.csv 

In [4]:
nj_mayors = load_file("../2022-nj-mayors-sample.csv")
nj_mayors.shape

(462, 9)

In [5]:
nj_mayors

,first_name,middle_name,last_name,house_number,street_address,city,state,zip_code,ZEST_KEY
0,Gabe,NaN,Plumer,782,Frenchtown Road,Milford,NJ,08848,2
1,Ari,NaN,Bernstein,500,West Crescent Avenue,Allendale,NJ,07401,4
2,David,J.,Mclaughlin,125,Corlies Avenue,Allenhurst,NJ,07711-1049,5
3,Thomas,C.,Fritts,8,North Main Street,Allentown,NJ,08501-1607,6
4,P.,NaN,McCkelvey,49,South Greenwich Street,Alloway,NJ,08001-0425,7
...,...,...,...,...,...,...,...,...,...
457,William,NaN,Degroff,3943,Route,Chatsworth,NJ,08019,558
458,Joseph,NaN,Chukwueke,200,Cooper Avenue,Woodlynne,NJ,08107-2108,559
459,Paul,NaN,Sarlo,85,Humboldt Street,Wood-Ridge,NJ,07075-2344,560
460,Craig,NaN,Frederick,120,Village Green Drive,Woolwich Township,NJ,08085-3180,562


#### Create output folder


In [6]:
make_directory()

Directory already exists


## Process Input Data
This step processes and standarizes the input data

In [7]:
from zrp.prepare.preprocessing import ProcessGeo

In [8]:
pg = ProcessGeo()
pg.fit(nj_mayors)
processed = pg.transform(nj_mayors, processed=False, replicate=False)

   [Start] Validating input geo data
   [Completed] Validating input geo data
   [Start] Processing geo data
      ...formatting


  0%|          | 0/462 [00:00<?, ?it/s][Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 15 concurrent workers.
[Parallel(n_jobs=-1)]: Done  20 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 170 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 420 tasks      | elapsed:    0.0s
100%|██████████| 462/462 [00:00<00:00, 15401.85it/s]

      ...address cleaning
      ...formatting
   [Completed] Processing geo data



[Parallel(n_jobs=-1)]: Done 462 out of 462 | elapsed:    0.0s finished


In [9]:
processed.head()

,ZEST_KEY,first_name,middle_name,last_name,house_number,street_address,city,state,zip_code,ZEST_KEY_COL,replicate_flg
0,2,GABE,None,PLUMER,782,FRENCHTOWN ROAD,MILFORD,NJ,08848,2,000000
1,4,ARI,None,BERNSTEIN,500,WEST CRESCENT AVENUE,ALLENDALE,NJ,07401,4,000000
2,5,DAVID,J,MCLAUGHLIN,125,CORLIES AVENUE,ALLENHURST,NJ,07711,5,000000
3,6,THOMAS,C,FRITTS,8,NORTH MAIN STREET,ALLENTOWN,NJ,08501,6,000000
4,7,P,None,MCCKELVEY,49,SOUTH GREENWICH STREET,ALLOWAY,NJ,08001,7,000000


#### Modeling Data Prep  
To integrate ACS data we will use `ACSModelPrep` 

Input data into the `ACSModelPrep` is expected to be tabluar data with the following columns: first name, middle name, last name, block group, census tract, zip code, and zest key. The `ZEST_KEY` should be specified to establish correspondence between inputs and outputs; it's effectively used as an index for the data table. Since we only want to use zip codes, we will have to bend the rules a bit with additional processing.

`ACSModelPrep` requires at least one Census GEOID block group, census tract, or zip code. In this example `zip_code` serves as the GEOID.


In [10]:
from zrp.prepare.prepare import ACSModelPrep

In [11]:
%%time
acs = ACSModelPrep()
acs.fit()

CPU times: user 17 µs, sys: 20 µs, total: 37 µs
Wall time: 44.3 µs


When called, the `.transform()` function's processing steps can include processing input data and integrating ACS data. 
- No data is out because `save_table` is set to False. If True then the data will be saved to a file 


In [12]:
zrp_output = acs.transform(processed, save_table=False)

Generating Geo IDs
   ...loading ACS lookup tables
   ... combining ACS & user input data
 ...Copy dataframes
 ...Block group
 ...Zip code
 ...No match
 ...Merge
 ...Merging complete


Working around defaults add placeholders for missing geographies

In [14]:
zrp_output[['GEOID_BG', 'GEOID_CT']] = None

### Invoke the ZRP_Predict on the sample data
To generate predictions, provide the path to the preferred pipeline directory in the `__init__`. Here we provide the path from the installed zrp package in the virtual environment.

If using the git cloned version of ZRP pipe path is: <br>
> `curpath = os.getcwd()`<br>
> `pipe_path = join(curpath, "../../zrp/modeling/models")`

In [15]:
from zrp.modeling.predict import ZRP_Predict_ZipCode
pipe_path = os.path.join(home, ".conda/envs/zrp_v0.3.3/lib/python3.7/site-packages/zrp/modeling/models")

Load feature list

In [16]:
feature_list = load_json(pipe_path + "/../feature_list_zp.json")

Generate Predictions

In [17]:
z_predict = ZRP_Predict_ZipCode(pipe_path= pipe_path)
z_predict.fit()
predict_out = z_predict.transform(zrp_output[feature_list].set_index("ZEST_KEY"))

100%|██████████| 1/1 [00:00<00:00, 1216.80it/s]
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.0s finished


#### Troubleshooting
If you run into a <font color='red'>FileNotFoundError</font>, it is likely that an incorrect `pipe_path` was provided.

The pipelines are downloaded upon installation when `python -m zrp download` is ran in the terminal. Please refer to the ReadMe if you missed this step. Otherwise, the current notebook is configured to run when the pipeline is downloaded to the package folder in your virtual environment. To check if the data is downloaded to the package folder in your virtual environment run the following command.

In [18]:
!find {pipe_path}  -print | grep -i 'pipe.pkl'

/home/kam/.conda/envs/zrp_v0.3.3/lib/python3.7/site-packages/zrp/modeling/models/block_group/pipe.pkl
/home/kam/.conda/envs/zrp_v0.3.3/lib/python3.7/site-packages/zrp/modeling/models/census_tract/pipe.pkl
/home/kam/.conda/envs/zrp_v0.3.3/lib/python3.7/site-packages/zrp/modeling/models/zip_code/pipe.pkl


The expected output will be similar to the following 

If`!find {pipe_path}  -print | grep -i 'pipe.pkl'` returns nothing, then update your `pipe_path` to point to your local git cloned zrp folder. An example is included below:
- `pipe_path = os.path.join(home, "zrp/zrp/modeling/models")`


### Inspect the output
- Preview the data


In [19]:
predict_out.head()

,AAPI,AIAN,BLACK,HISPANIC,WHITE,race_proxy,source_zrp_zip_code
ZEST_KEY,,,,,,,
555,0.002544,0.024130,0.160289,0.006178,0.806859,WHITE,1
556,0.000413,0.000537,0.002492,0.906133,0.090425,HISPANIC,1
557,0.012433,0.002701,0.038109,0.022682,0.924076,WHITE,1
558,0.000475,0.000497,0.002019,0.001039,0.995970,WHITE,1
559,0.008601,0.001355,0.901341,0.037128,0.051575,BLACK,1


Note: Source columns denote which method was used to generate proxies.